## Agenda
- Extract the data from spotify api (top songs - global)

> note: this playlist will update every week.

External lib <br>
- pip install spotipy
- pip install pandas

In [1]:
!pip install pandas

In [2]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import json
import os

In [3]:
client_credentials_manager = SpotifyClientCredentials(client_id=os.environ['client_id'], client_secret=os.environ['client_secret'])

In [4]:
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [5]:
playlist_link='https://open.spotify.com/playlist/37i9dQZEVXbNG2KDcFcKOF'

In [6]:
playlist_URI=playlist_link.split("/")[-1]
print(playlist_URI)

37i9dQZEVXbNG2KDcFcKOF


In [7]:
data = sp.playlist_tracks(playlist_URI)

print(data)

{'href': 'https://api.spotify.com/v1/playlists/37i9dQZEVXbNG2KDcFcKOF/tracks?offset=0&limit=100&additional_types=track', 'items': [{'added_at': '2024-04-26T11:35:50Z', 'added_by': {'external_urls': {'spotify': 'https://open.spotify.com/user/'}, 'href': 'https://api.spotify.com/v1/users/', 'id': '', 'type': 'user', 'uri': 'spotify:user:'}, 'is_local': False, 'primary_color': None, 'track': {'preview_url': None, 'available_markets': ['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA', 'CL', 'CO', 'CR', 'CY', 'CZ', 'DK', 'DO', 'DE', 'EC', 'EE', 'SV', 'FI', 'FR', 'GR', 'GT', 'HN', 'HK', 'HU', 'IS', 'IE', 'IT', 'LV', 'LT', 'LU', 'MY', 'MT', 'MX', 'NL', 'NZ', 'NI', 'NO', 'PA', 'PY', 'PE', 'PH', 'PL', 'PT', 'SG', 'SK', 'ES', 'SE', 'CH', 'TW', 'TR', 'UY', 'US', 'GB', 'AD', 'LI', 'MC', 'ID', 'JP', 'TH', 'VN', 'RO', 'IL', 'ZA', 'SA', 'AE', 'BH', 'QA', 'OM', 'KW', 'EG', 'MA', 'DZ', 'TN', 'LB', 'JO', 'PS', 'IN', 'KZ', 'MD', 'UA', 'AL', 'BA', 'HR', 'ME', 'MK', 'RS', 'SI', 'KR', 'BD', 'PK', 'LK', 'GH', 

## Convert data to JSON format

In [8]:
# Convert data to JSON format
json_data = json.dumps(data)

# Save JSON data to a file
with open('data.json', 'w') as f:
    f.write(json_data)

In [9]:
album_list = []
for row in data['items']:
    album_id = row['track']['album']['id']
    album_name = row['track']['album']['name']
    album_release_date = row['track']['album']['release_date']
    album_total_tracks = row['track']['album']['total_tracks']
    album_url = row['track']['album']['external_urls']['spotify']
    album_element = {'album_id':album_id,'name':album_name,'release_date':album_release_date,
                        'total_tracks':album_total_tracks,'url':album_url}
    album_list.append(album_element)

In [10]:
artist_list = []
for row in data['items']:
    for key, value in row.items():
        if key == "track":
            for artist in value['artists']:
                artist_dict = {'artist_id':artist['id'], 'artist_name':artist['name'], 'external_url': artist['href']}
                artist_list.append(artist_dict)

In [11]:
song_list = []
for row in data['items']:
    song_id = row['track']['id']
    song_name = row['track']['name']
    song_duration = row['track']['duration_ms']
    song_url = row['track']['external_urls']['spotify']
    song_popularity = row['track']['popularity']
    song_added = row['added_at']
    album_id = row['track']['album']['id']
    artist_id = row['track']['album']['artists'][0]['id']
    song_element = {'song_id':song_id,'song_name':song_name,'duration_ms':song_duration,'url':song_url,
                    'popularity':song_popularity,'song_added':song_added,'album_id':album_id,
                    'artist_id':artist_id
                   }
    song_list.append(song_element)

## Album table data frame

In [12]:
album_df = pd.DataFrame.from_dict(album_list)
# display(album_df)

In [13]:
album_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   album_id      50 non-null     object
 1   name          50 non-null     object
 2   release_date  50 non-null     object
 3   total_tracks  50 non-null     int64 
 4   url           50 non-null     object
dtypes: int64(1), object(4)
memory usage: 2.1+ KB


### Remove duplicates

In [14]:
album_df = album_df.drop_duplicates(subset=['album_id'])
display(album_df)

,album_id,name,release_date,total_tracks,url
0,1Mo4aZ8pdj6L1jx8zSwJnt,THE TORTURED POETS DEPARTMENT,2024-04-18,16,https://open.spotify.com/album/1Mo4aZ8pdj6L1jx...
2,5HIWDdg3g9CTOtnevKDl1z,i like the way you kiss me,2024-03-19,1,https://open.spotify.com/album/5HIWDdg3g9CTOtn...
7,5tSQtQGkrCJx3hoQxmLgfM,Gata Only,2024-02-02,1,https://open.spotify.com/album/5tSQtQGkrCJx3ho...
10,3P4SQqMMgjqfqVxgLwtYRk,Unreal Unearth: Unheard,2024-03-22,20,https://open.spotify.com/album/3P4SQqMMgjqfqVx...
12,5quMTd5zeI9yW5UDua8wS4,Espresso,2024-04-12,1,https://open.spotify.com/album/5quMTd5zeI9yW5U...
14,168CdR21lfn0TTyw1Pkdcm,Fireworks & Rollerblades,2024-04-05,15,https://open.spotify.com/album/168CdR21lfn0TTy...
15,4JDXBZdRPId4devseaAOKH,DECIDE,2022-09-16,13,https://open.spotify.com/album/4JDXBZdRPId4dev...
16,5EYKrEDnKhhcNxGedaRQeK,eternal sunshine,2024-03-08,13,https://open.spotify.com/album/5EYKrEDnKhhcNxG...
23,3UOV8XvCwMKaATRNXrYCjN,greedy,2023-09-15,1,https://open.spotify.com/album/3UOV8XvCwMKaATR...
24,7nacKlk586eLRBSIsrk9DB,I've Tried Everything But Therapy (Part 1),2023-09-15,10,https://open.spotify.com/album/7nacKlk586eLRBS...


In [15]:
album_df['release_date'] = pd.to_datetime(album_df['release_date'])

In [16]:
album_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 26 entries, 0 to 49
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   album_id      26 non-null     object        
 1   name          26 non-null     object        
 2   release_date  26 non-null     datetime64[ns]
 3   total_tracks  26 non-null     int64         
 4   url           26 non-null     object        
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 1.2+ KB


## Artist DF

In [17]:
artist_df = pd.DataFrame.from_dict(artist_list)

In [18]:
artist_df = artist_df.drop_duplicates(subset=['artist_id'])

In [19]:
artist_df.head()

,artist_id,artist_name,external_url
0,06HL4z0CvFAxyc27GXpf02,Taylor Swift,https://api.spotify.com/v1/artists/06HL4z0CvFA...
1,246dkjvS1zLTtiykXe5h60,Post Malone,https://api.spotify.com/v1/artists/246dkjvS1zL...
3,0PCCGZ0wGLizHt2KZ7hhA2,Artemas,https://api.spotify.com/v1/artists/0PCCGZ0wGLi...
8,7CvTknweLr9feJtRGrpDBy,FloyyMenor,https://api.spotify.com/v1/artists/7CvTknweLr9...
9,1Yj5Xey7kTwvZla8sqdsdE,Cris Mj,https://api.spotify.com/v1/artists/1Yj5Xey7kTw...


## Song Dataframe

In [20]:
song_df = pd.DataFrame.from_dict(song_list)

In [21]:
song_df.head()

,song_id,song_name,duration_ms,url,popularity,song_added,album_id,artist_id
0,2OzhQlSqBEmt7hmkYxfT6m,Fortnight (feat. Post Malone),228965,https://open.spotify.com/track/2OzhQlSqBEmt7hm...,93,2024-04-26T11:35:50Z,1Mo4aZ8pdj6L1jx8zSwJnt,06HL4z0CvFAxyc27GXpf02
1,2F3N9tdombb64aW6VtZOdo,Down Bad,261228,https://open.spotify.com/track/2F3N9tdombb64aW...,87,2024-04-26T11:35:50Z,1Mo4aZ8pdj6L1jx8zSwJnt,06HL4z0CvFAxyc27GXpf02
2,2GxrNKugF82CnoRFbQfzPf,i like the way you kiss me,142514,https://open.spotify.com/track/2GxrNKugF82CnoR...,100,2024-04-26T11:35:50Z,5HIWDdg3g9CTOtnevKDl1z,0PCCGZ0wGLizHt2KZ7hhA2
3,3Vevii7qKqrmW8CcyzBHDl,"So Long, London",262974,https://open.spotify.com/track/3Vevii7qKqrmW8C...,85,2024-04-26T11:35:50Z,1Mo4aZ8pdj6L1jx8zSwJnt,06HL4z0CvFAxyc27GXpf02
4,4q5YezDOIPcoLr8R81x9qy,I Can Do It With a Broken Heart,218004,https://open.spotify.com/track/4q5YezDOIPcoLr8...,86,2024-04-26T11:35:50Z,1Mo4aZ8pdj6L1jx8zSwJnt,06HL4z0CvFAxyc27GXpf02


In [22]:
song_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   song_id      50 non-null     object
 1   song_name    50 non-null     object
 2   duration_ms  50 non-null     int64 
 3   url          50 non-null     object
 4   popularity   50 non-null     int64 
 5   song_added   50 non-null     object
 6   album_id     50 non-null     object
 7   artist_id    50 non-null     object
dtypes: int64(2), object(6)
memory usage: 3.3+ KB


In [23]:
song_df['song_added'] =  pd.to_datetime(song_df['song_added'])

In [24]:
song_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype              
---  ------       --------------  -----              
 0   song_id      50 non-null     object             
 1   song_name    50 non-null     object             
 2   duration_ms  50 non-null     int64              
 3   url          50 non-null     object             
 4   popularity   50 non-null     int64              
 5   song_added   50 non-null     datetime64[ns, UTC]
 6   album_id     50 non-null     object             
 7   artist_id    50 non-null     object             
dtypes: datetime64[ns, UTC](1), int64(2), object(5)
memory usage: 3.3+ KB
